Osnabrück University - Computer Vision (Winter Term 2020/21) - Prof. Dr.-Ing. G. Heidemann, Ulf Krumnack, Ludwig Schallner, Artem Petrov

# Exercise Sheet 8: Compression

## Introduction

This week's sheet should be solved and handed in before the end of **Saturday, January 09, 2021**. If you need help (and Google and other resources were not enough), feel free to contact your groups' designated tutor or whomever of us you run into first. Please upload your results to your group's Stud.IP folder.

## Assignment 1: Redundancy and compression [4 Points]


**a)** Explain in your own words the different types of redundancy mentioned on (CV-10 slide 3). How can you check for each of these types of redundancy?

Coding redundancy occurs when an encoding is able to represent more information than it has to represent. Thus, the used encoding covers more codes than necessary, so it is "overpowered" for that particular usecase.

Interpixel redundancy occurs when neighboring pixels have the same gray/ color value. So there is potential for compression by combining these larger pixel areas of the same gray/ color value and encoding them as a single entity. 

Psychovisual redundancy describes the effect that an image contains more information than the human eye ia able to recognize. Hence, some information can be removed due to compression reasons without disturbance of the human's eye perception.  

**b)** Explain the differences between lossless and lossy compression. Name examples for both of them. Sketch application scenarios.

A lossless compression compresses data in a way that keeps all information given in the original image. -> e.g. Huffman Decoding, Run Length Encoding, Gray code

A lossy compression compresses an image by removing some of the information given in the original image. 

## Assignment 2: Entropy based compression [8 Points]


**a)** Explain the idea of Huffman coding. What is the maximal compression factor that can be achieved for a given image? Load an image and compute that value (you may use `dolly.png` as an example. Make sure to load as 8-bit gray scale image).

The idea of Huffman coding is to encode the different gray values of an image according to their frequency. Thus, the gray values with the most frequent occurence are encoded with the smallest codes. 

Maximum compression factor: n_bits / entropy

In [6]:
import numpy as np
import math
import imageio
import matplotlib.pyplot as plt

img = imageio.imread('images/dolly.png', pilmode='L')

# YOUR CODE HERE
def entropy(img):
    """
    Compute the entropy for a given image.
    Args:
        img (ndarray): The grayscale image to compute the entropy for.
    
    Returns:
        img_entropy (float): The entropy of the image. 
    """
    # YOUR CODE HERE
    n_gray_values = 256
    frequency_each_gray_value = np.zeros(n_gray_values)  #saves the frequency of each gray value in the image (index = corresponding gray value)
    for img_row in range(img.shape[0]):
        for img_col in range(img.shape[1]):
            #Increments the count in the frequency array at the index which fits the current gray value of the image.
            frequency_each_gray_value[img[img_row][img_col]] += 1    
            
    #Computes the probability of each gray value -> "Normalization"
    probability_each_gray_value = frequency_each_gray_value / (img.shape[0] * img.shape[1])
    
    entropy = 0
    base = 2   #base of log
    for gray_value in probability_each_gray_value:   #range for. Only applicable if the array, over which iteration is performed, is not edited. Because range-for works on a copy of the original array, so changes to the array are not saved. 
        #log is only defined for values larger than 0
        entropy += gray_value * math.log(gray_value if gray_value > 0 else 1, base)
    
    return (entropy * (-1))

n_bits = 8
maximum_compression_factor = n_bits / entropy(img)

print("Entropy: " + str(entropy(img)))
print("Maximum compression factor: " + str(maximum_compression_factor))

Entropy: 7.426579595591895
Maximum compression factor: 1.0772119112206733


**b)** Now compute the relative frequencies (normalized histogram) of the image and generate an (approximately) balanced tree, as described in (CV-10 slide 6). *Hint:* you may use Python tuples as building blocks of a tree. Every non-leaf node is a pair `(left, right)` where `left` and `right` are the left and right subtrees, respectively (of course you are free to choose another approach if you prefer to do so).

In [28]:
from collections import namedtuple
from sortedcontainers import SortedList  

Node = namedtuple('Node', ['frequency', 'subtree'])

img = imageio.imread('images/dolly.png', pilmode='L')

occurences = np.zeros(256, dtype=np.int32)
for img_row in range(img.shape[0]):
    for img_col in range(img.shape[1]):
        occurences[img[img_row][img_col]] += 1
#print(occurences)

n_pixels = img.shape[0] * img.shape[1]
normed_hist_img = np.zeros_like(occurences, dtype=np.float32)
for i in range(len(occurences)):
    normed_hist_img[i] = (occurences[i] / n_pixels)             # normalized histogram
#print(normed_hist_img)

# nodes is a sorted list of sub-trees, each annotated by its cummulative relative frequency,
# i.e. each list item is a pair (frequency, subtree)
# lowest frequencies come first
nodes = SortedList(
    [Node(frequency, i) for i, frequency in enumerate(normed_hist_img)],
    key=lambda x: x.frequency)


# YOUR CODE HERE
#print(nodes)
first_non_leaf = nodes[0][0] + nodes[1][0]
print(first_non_leaf)
current_least_likely_symbols = 0
for i in range(len(nodes)-1):
    current_least_likely_symbols += nodes[i][0]
    current_non_leaf_pair = (current_least_likely_symbols, nodes[i+1][0])
    print(current_non_leaf_pair)





#print(tree) Wie in einen tree packen???

0.0
(0.0, 0.0)
(0.0, 8.333333e-06)
(8.333333425980527e-06, 1.6666667e-05)
(2.500000027794158e-05, 2.7083333e-05)
(5.208333368500462e-05, 2.9166667e-05)
(8.125000113068381e-05, 2.9166667e-05)
(0.00011041666857636301, 3.5416666e-05)
(0.00014583333449991187, 3.5416666e-05)
(0.00018125000042346073, 4.375e-05)
(0.00022499999886349542, 4.5833334e-05)
(0.00027083333316113567, 4.5833334e-05)
(0.0003166666674587759, 4.5833334e-05)
(0.00036250000175641617, 6.875e-05)
(0.00043125000138388714, 0.00013541666)
(0.0005666666611432447, 0.00014166666)
(0.0007083333248374402, 0.00015416667)
(0.0008624999964013114, 0.00017916667)
(0.0010416666691526189, 0.00018958333)
(0.0012312499957261025, 0.0002125)
(0.0014437499985433533, 0.00027916668)
(0.001722916676044406, 0.00039791668)
(0.002120833355547802, 0.000425)
(0.0025458333611823036, 0.00052291667)
(0.003068750032070966, 0.00054166664)
(0.0036104166711083963, 0.00071041664)
(0.004320833314523043, 0.0007166667)
(0.005037500005528273, 0.00073125)
(0.005768

NameError: name 'tree' is not defined

**c)** Now create a prefix free code from this tree, by traversing it following the idea sketched in (CV-10 slide 7). *Hint:* if you used the tuple representation recommended in (b), you can use `isinstance(node, tuple)` to check if `node` is an inner node or a leaf.

In [ ]:
# initialize a list of code values
codes = list(normed_hist_img)

# function to recursively traverse the tree.
# For every inner node assign prefix "0" to the left subtree
# and prefix "1" to the right subtree.
def assign_codes(codes, tree, prefix=''):
    # YOUR CODE HERE

assign_codes(codes, tree.subtree)
print(codes)

**d)** Compute the compression ratio that you can achieve with that code. Compare this with the maximal value computed in part (a). Explain your observation.

In [ ]:
acc = 0
for value, code in enumerate(codes):
    # YOUR CODE HERE

print("{:.2f} bits per pixel".format(acc))

YOUR ANSWER HERE

## Assignment 3: Run length encoding [8 Points]


**a)** Explain the idea of *run length encoding*. What are advantages and disadvantages? In what situations should it be applied?

Run Length Encoding (RLE) combines and encodes neighboring pixels as one segment if they have the identical gray/ color value. The encoding is conducted line by line. The segments are encoded in the form (number, value), so in the worst case, if there are no segments with same gray/ color values, the amount of data is doubled.

**b)** Analyze the run lengths in a gray scale image (8 bit = 256 gray values) by counting the number of runs and the average run length and displaying a histogram of the run lengths. What do you observe? Can you benefit from run length encoding here? (you may use `dolly.png` as an example again, but you may also experiment with other images. Make sure to load it as 8-bit gray scale image).

In [ ]:
import numpy as np
import imageio
import matplotlib.pyplot as plt

img = imageio.imread('images/dolly.png', pilmode='L')

# YOUR CODE HERE

YOUR ANSWER HERE

**c)** Now consider the individual bit planes. First display the bit planes as in (CV-10 slide 10ff). What do you observe? Apply your analysis from part (b) to each bitplane.

In [ ]:
import numpy as np
import imageio
import matplotlib.pyplot as plt

img = imageio.imread('images/dolly.png', pilmode='L')

# YOUR CODE HERE

YOUR ANSWER HERE

**d)** Explain the idea of the *Gray code*. Why is it better suited for run length encoding? Compute a Gray code for a 256 bit image and recode the image `dolly.png`. Then analyze the run lengths of the individual bit planes of the recoded image.

YOUR ANSWER HERE

In [ ]:
import numpy as np
import imageio
import matplotlib.pyplot as plt

img = imageio.imread('images/dolly.png', pilmode='L')

# YOUR CODE HERE